In [3]:
import os, json, csv, itertools
import numpy as np

from sklearn.preprocessing import normalize

In [4]:
users = open("C:\\Users\\barre\\Documents\\UFPE 2015\\TCC\\MovieLens\\basic\\users_basic_sample.csv").read().split('\n')

In [20]:
parent_dir = './Results/Surprise/mae/'

results_files = ['BaselineOnly(SGD).txt',
'CoClustering.txt',
'KNNBasic(Itembased).txt',
'KNNBasic(Userbased).txt',
'KNNWithMeans(Itembased).txt',
'KNNWithMeans(Userbased).txt',
'NMF.txt',
'NormalPredictor.txt',
'SlopeOne.txt',
'SVD.txt']

mae_results = 'mae_results.json'

answer_matrix = 'answer_matrix_mae_.csv'
time_matrix = 'time_matrix.csv'

In [6]:
len(results_files)

10

In [7]:
def read_results(root_dir):
    results = []
    for file in results_files:
        with open(os.path.join(root_dir, file)) as result:
            content = result.read().split('\n')[3:]
            results.append(json.loads(''.join(content)))
        result.close()
    return results

In [8]:
def get_time(root_dir):
    results = read_results(root_dir)
    time = []
    for user in users:
        time.append(list(map(lambda x: x[user]['time'], results)))
    return time

In [9]:
def write_file(filename, headers, rows):
    with open(os.path.join(parent_dir, filename), 'w', newline='') as file:
        file_writer = csv.writer(file)
        file_writer.writerow(headers)
        for row in rows:
            file_writer.writerow(row)
    file.close()

In [17]:
def without_scale(root_dir, metric='RMSE', results=None):
    if root_dir:
        results = read_results(root_dir)
    rmse = []
    for user in users:
        rmse.append(list(map(lambda x: x[user][metric], results)))
    return rmse

In [11]:
def unit_scale(root_dir, all_=False):
    results = read_results(root_dir)
    results_list = list(map(lambda x: list(map(lambda y: y[1]['RMSE'], x.items())), results))
    if all_:
        results_normalized = list(itertools.chain.from_iterable(results_list))
        results_normalized = 1-normalize([results_normalized])
        results_normalized = list(map(list, list(zip(*[iter(results_normalized[0])] * 200))))
    else:
        results_normalized = 1-normalize(results_list)
    return np.transpose(results_normalized)

In [12]:
def min_max_scale(root_dir, all_=True):
    results = read_results(root_dir)
    results_list = list(map(lambda x: list(map(lambda y: y[1]['RMSE'], x.items())), results))
    rmse = []
    if all_:
        big_list = list(itertools.chain.from_iterable(results_list))
        r_max = max(big_list)
        r_min = min(big_list)
        rmse = list(map(lambda x: (r_max - x)/(r_max - r_min), big_list))
    else:
        for result in results_list:
            r_max = max(result)
            r_min = min(result)
            rmse.append([(r_max - x)/(r_max - r_min) for x in result])
        rmse = list(itertools.chain.from_iterable(rmse))
    rmse = list(map(list, list(zip(*[iter(rmse)] * 200))))
    rmse = np.transpose(rmse)
    return rmse

In [13]:
def standard_scale(root_dir=None, metric='RMSE', results=None):
    if root_dir:
        results = read_results(root_dir)
        
    if results:        
        results_list = list(map(lambda x: list(map(lambda y: y[1][metric], x.items())), results))
        results_normalized = list(itertools.chain.from_iterable(results_list))
        results_normalized = list(map(lambda x: 1/(1+x), results_normalized))
        results_normalized = list(map(list, list(zip(*[iter(results_normalized)] * 200))))        
        return np.transpose(results_normalized)
    
    return None

In [14]:
def standard_scale2(results, metric='mae'):
    normal = list(map(lambda x: x[1][metric], results['NormalPredictor'].items()))
    results_list = []
    for alg, value in results.items():
        results_list_temp = list(map(lambda x: x[1][metric], value.items()))
        results_list.append(list(map(lambda x: 1/(1+(x[0]/x[1])), list(zip(results_list_temp, normal)) )))
        
    results_normalized = list(itertools.chain.from_iterable(results_list))
    results_normalized = list(map(list, list(zip(*[iter(results_normalized)] * 200))))        
    return np.transpose(results_normalized)

In [15]:
# headers = [file.replace('.txt', '') for file in results_files]
# rmse = standard_scale(parent_dir)
# write_file(answer_matrix, headers, rmse)
# write_file(time_matrix, headers, time)

In [26]:
mae_results_file = open(parent_dir+mae_results)
results = json.load(mae_results_file)
headers = list(results.keys())
mae = standard_scale(metric='mae', results=[value for _, value in results.items()])
# mae = without_scale(root_dir=None, metric='mae', results=[value for _, value in results.items()])
write_file(answer_matrix, headers, mae)